In [4]:
import tensorflow as tf
import numpy as np
import os
import sys
import pandas
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import keras
import shutil
import random
import pathlib
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models,layers,optimizers
#必要的库和库的版本检查#
print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pandas,sklearn,tf,keras:
    print(module.__name__, )

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib
numpy
pandas
sklearn
tensorflow
tensorflow_core.keras


In [6]:
#取数据集#
cifar10=keras.datasets.cifar10
(x_all,y_all),(x_test,y_test)=cifar10.load_data()
x_val,x_train=x_all[:5000],x_all[5000:]
y_val,y_train=y_all[:5000],y_all[5000:]

print(x_val.shape,y_val.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

170500096/170498071 [==============================] - 1306s 8us/step
(5000, 32, 32, 3) (5000, 1)
(45000, 32, 32, 3) (45000, 1)
(10000, 32, 32, 3) (10000, 1)


In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,32,32,3)
x_val_scaled=scaler.fit_transform(x_val.astype(np.float32).reshape(-1,1)).reshape(-1,32,32,3)
x_test_scaled=scaler.fit_transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,32,32,3)
print("ok")

ok


In [10]:
model=keras.models.Sequential()
flt=64
model.add(tf.keras.layers.SeparableConv2D(filters=32,kernel_size=3,padding="same",activation="selu",input_shape=(32,32,3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=2))
model.add(tf.keras.layers.SeparableConv2D(filters=flt,kernel_size=3,padding="same",activation="selu"))
model.add(tf.keras.layers.SeparableConv2D(filters=flt,kernel_size=3,padding="same",activation="selu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2))
model.add(tf.keras.layers.SeparableConv2D(filters=flt,kernel_size=3,padding="same",activation="selu"))
model.add(tf.keras.layers.SeparableConv2D(filters=flt,kernel_size=3,padding="same",activation="selu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024,activation='relu'))
model.add(tf.keras.layers.Dense(1000,activation='relu'))
model.add(keras.layers.Dense(128,activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",optimizer="sgd",metrics=["accuracy"])
model.summary()
print("ok")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_10 (Separab (None, 32, 32, 32)        155       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
separable_conv2d_11 (Separab (None, 16, 16, 64)        2400      
_________________________________________________________________
separable_conv2d_12 (Separab (None, 16, 16, 64)        4736      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
separable_conv2d_13 (Separab (None, 8, 8, 64)          4736      
_________________________________________________________________
separable_conv2d_14 (Separab (None, 8, 8, 64)         

In [14]:
config = tf.config
config.gpu_options.allow_growth = True
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))
logdir=os.path.join("cifar10")
if not os.path.exists(logdir):
    os.mkdir(logdir)
outputs=os.path.join(logdir,"cifar10_model.h5")
callbacks=[keras.callbacks.TensorBoard(logdir),
           keras.callbacks.ModelCheckpoint(outputs,save_best_only=True),
           keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)]


def train(model):
    history=model.fit(x_train_scaled,y_train,epochs=14,validation_data=(x_val_scaled,y_val),callbacks=callbacks)
    return history
histories=train(model)

AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'gpu_options'

In [ ]:
def showDnCurve(history):
    pandas.DataFrame(history.history).plot(figsize=(10,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1.5)
    plt.show()
showDnCurve(histories)